# 5. Arrays

In [148]:
from typing import List, Iterator, Dict
import math
import collections
from collections import Counter
import numpy as np

In [2]:
def even_odd(A: List[int]) -> List[int]:
    next_even, next_odd = 0, len(A) - 1
    while next_even < next_odd:
        if A[next_even] % 2 == 0:
            next_even += 1
        else:
            A[next_even], A[next_odd] = A[next_odd], A[next_even]
            next_odd -= 1
    return A

In [3]:
even_odd([1, 2, 3, 4, 5])

[4, 2, 3, 5, 1]

In [156]:
def dutch_flag_partition(pivot_index: int, A: List[int]) -> None:
    pivot = A[pivot_index]
    # First pass: group elements smaller than pivot.
    smaller = 0
    for i in range(len(A)):
        if A[i] < pivot:
            A[i], A[smaller] = A[smaller], A[i]
            smaller += 1
    # Second pass: group elements larger than pivot.
    larger = len(A) - 1
    for i in reversed(range(len(A))):
        if A[i] > pivot:
            A[i], A[larger] = A[larger], A[i]
            larger -= 1

In [157]:
%timeit dutch_flag_partition(5000, list(reversed(range(10000))))

4 ms ± 1.12 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
def dutch_flag_partition(pivot_index: int, A: List[int]) -> None:
    pivot = A[pivot_index]
    # Keep the following invariants during partitioning:
    # bottom group: A[:smaller].
    # middle group: A[smaller:equal].
    # unclassified group: A[equal:larger]
    # top group: A[]
    smaller, equal, larger = 0, 0, len(A)
    # Keep iterating as long as there is an unclassified element
    while equal < larger:
        if A[equal] < pivot:
            A[smaller], A[equal] = A[equal], A[smaller]
            smaller, equal = smaller + 1, equal + 1
        elif A[equal] == pivot:
            equal += 1
        else:
            larger -= 1
            A[equal], A[larger] = A[larger], A[equal]

In [7]:
%timeit dutch_flag_partition(5000, list(reversed(range(10000))))

3.16 ms ± 13.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
def plus_one(A: List[int]) -> List[int]:
    A[-1] += 1
    for i in reversed(range(1, len(A))):
        if A[i] != 10:
            break
        A[i] = 0
        A[i - 1] += 1
    if A[0] == 10:
        # There is a carry-out, so we need one more digit to store the result.
        # A lick way to do this is to append a 0 at the end of the array,
        # and update the first entry to 1.
        A[0] = 1
        A.append(0)
    return A

In [9]:
%timeit plus_one([9] * 10000)

2.23 ms ± 22.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
def multiply(num1: List[int], num2: List[int]) -> List[int]:
    sign = -1 if (num1[0] < 0) ^ (num2[0] < 0) else 1
    num1[0], num2[0] = abs(num1[0]), abs(num2[0])
    
    result = [0] * (len(num1) + len(num2))
    for i in reversed(range(len(num1))):
        for j in reversed(range(len(num2))):
            result[i + j + 1] += num1[i] * num2[j]
            result[i + j] += result[i + j + 1] // 10
            result[i + j + 1] %= 10
            
    # Remove the leading zeroes
    result = result[next((i for i, x in enumerate(result) if x != 0), len(result)):] or [0]
    return [sign * result[0]] + result[1:]

In [11]:
multiply([-5], [-3, 2])

[1, 6, 0]

In [12]:
def can_reach_end(A: List[int]) -> bool:
    furthest_reach_so_far, last_index = 0, len(A) - 1
    i = 0
    while i <= furthest_reach_so_far and furthest_reach_so_far < last_index:
        furthest_reach_so_far = max(furthest_reach_so_far, A[i] + i)
        i += 1
    return furthest_reach_so_far >= last_index

In [13]:
can_reach_end([3, 3, 1, 0, 2, 0, 1])

True

In [14]:
can_reach_end([3, 2, 0, 0, 2, 0, 1])

False

In [15]:
# Return the number of valid entries after deletion.
def delete_duplicates(A: List[int]) -> int:
    if not A:
        return []
    write_index = 1
    for i in range(1, len(A)):
        if A[write_index - 1] != A[i]:
            A[write_index] = A[i]
            write_index += 1
    return A[:write_index]

In [16]:
delete_duplicates([2, 3, 5, 5, 7, 11, 11, 11, 13])

[2, 3, 5, 7, 11, 13]

In [17]:
def buy_and_sell_stock_once(prices: List[float]) -> float:
    min_price_so_far, max_profit = float('inf'), 0.0
    for price in prices:
        max_profit_sell_today = price - min_price_so_far
        max_profit = max(max_profit, max_profit_sell_today)
        min_price_so_far = min(min_price_so_far, price)
    return max_profit

In [18]:
buy_and_sell_stock_once([310, 315, 275, 295, 260, 270, 290, 230, 255, 250])

30

In [19]:
def buy_and_sell_stock_twice(prices: List[float]) -> float:
    max_total_profit, min_price_so_far = 0.0, float('inf')
    first_buy_sell_profits = [0.0] * len(prices)
    # Forward phase. For each day, we record maximum profit if we sell on that day.
    for i, price in enumerate(prices):
        min_price_so_far = min(min_price_so_far, price)
        max_total_profit = max(max_total_profit, price - min_price_so_far)
        first_buy_sell_profits[i] = max_total_profit
    max_price_so_far = float('-inf')
    # Backward phase. For each day, find the maximum profit if we make the 
    # second buy on that day
    for i, price in reversed(list(enumerate(prices[1:], 1))):
        max_price_so_far = max(max_price_so_far, price)
        max_total_profit = max(max_total_profit, 
                               max_price_so_far - price + first_buy_sell_profits[i - 1])
    return max_total_profit

In [20]:
buy_and_sell_stock_twice([310, 315, 275, 295, 260, 270, 290, 230, 255, 250])

55

In [21]:
def buy_and_sell_stock_twice(prices: List[float]) -> float:
    min_price = float('inf')
    max_profit_after_first_sell = 0
    max_profit_left_after_second_buy = float('-inf')
    max_profit_after_second_sell = 0
    for price in prices:
        min_price = min(price, min_price)
        max_profit_after_first_sell = max(price - min_price, max_profit_after_first_sell)
        max_profit_left_after_second_buy = max(max_profit_after_first_sell - price, max_profit_left_after_second_buy)
        max_profit_after_second_sell = max(price + max_profit_left_after_second_buy, max_profit_after_second_sell)
    return max_profit_after_second_sell

In [22]:
buy_and_sell_stock_twice([310, 315, 275, 295, 260, 270, 290, 230, 255, 250])

55

In [23]:
def rearrange(A: List[int]) -> List[int]:
    for i in range(len(A)):
        A[i:i + 2] = sorted(A[i:i + 2], reverse=bool(i % 2))
    return A

In [24]:
%timeit rearrange(list(range(10000)))

9.58 ms ± 3.55 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
# Given n, return all primes up to and including n.
def generate_primes(n: int) -> List[int]:
    primes = []
    # is_prime[p] represents if p is prime or not. Initially, set each to 
    # true, expecting 0 and 1. Then use sieving to eliminate nonprimes.
    is_prime = [False, False] + [True] * (n - 1)
    for p in range(2, n + 1):
        if is_prime[p]:
            primes.append(p)
            # Sieve p's multiples.
            for i in range(p * 2, n + 1, p):
                is_prime[i] = False
    return primes

In [26]:
%timeit generate_primes(10000)

3.08 ms ± 1.15 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
# Given n, return all primes up to and including n.
def generate_primes(n: int) -> List[int]:
    if n < 2:
        return []
    size = (n - 3) // 2 + 1
    primes = [2]  # Stores the primes from 1 to n.
    # is_prime[i] represents (2i + 3) is prime or not.
    # For example, is_prime[0] represents 3 is prime or not,
    # is_prime[1] represents 5, is_prime[2] represents 7, etc.
    # Initially set each to true. Then use sieving to eliminate nonprimes.
    is_prime = [True] * size
    for i in range(size):
        if is_prime[i]:
            p = i * 2 + 3
            primes.append(p)
            # Sieving from p^2, where p^2 = (4i^2 + 12i + 9).
            # The index in is_prime is (2i^2 + 6i + 3)
            for j in range(2 * i**2 + 6 * i + 3, size, p):
                is_prime[j] = False
    return primes

In [28]:
%timeit generate_primes(10000)

1.82 ms ± 233 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [39]:
def apply_permutation(perm: List[int], A: List[int]) -> List[int]:
    for i in range(len(A)):
        while perm[i] != i:
            A[perm[i]], A[i] = A[i], A[perm[i]]
            perm[perm[i]], perm[i] = perm[i], perm[perm[i]]
    print(perm)
    return A

In [40]:
apply_permutation([2, 0, 1, 4, 5, 3], [10, 11, 12, 13, 14, 15])

[0, 1, 2, 3, 4, 5]


[11, 12, 10, 15, 13, 14]

In [37]:
def apply_permutation(perm: List[int], A: List[int]) -> List[int]:
    for i in range(len(A)):
        if perm[i] < 0:
            perm[i] = -perm[i] - 1
            continue
        a, j = A[i], perm[i]
        while j != i:
            A[j], a = a, A[j]
            perm[j], j = -perm[j] - 1, perm[j]
        A[i] = a
    print(perm)
    return A

In [38]:
apply_permutation([2, 0, 1, 4, 5, 3], [10, 11, 12, 13, 14, 15])

[2, 0, 1, 4, 5, 3]


[11, 12, 10, 15, 13, 14]

In [41]:
def next_permutation(perm: List[int]) -> List[int]:
    inversion_point = len(perm) - 2
    while (inversion_point >= 0 and perm[inversion_point] >= perm[inversion_point + 1]):
        inversion_point -= 1
    if inversion_point == -1:
        return []
    for i in reversed(range(inversion_point + 1, len(perm))):
        if perm[i] > perm[inversion_point]:
            perm[inversion_point], perm[i] = perm[i], perm[inversion_point]
            break
    perm[inversion_point + 1:] = reversed(perm[inversion_point + 1:])
    return perm

In [43]:
next_permutation([2, 0, 1, 5, 4, 3])

[2, 0, 3, 1, 4, 5]

In [46]:
import random

def random_sampling(k: int, A: List[int]) -> List[int]:
    for i in range(k):
        r = random.randint(i, len(A) - 1)
        A[i], A[r] = A[r], A[i]
    return A

In [58]:
random_sampling(10, list(range(10)))

[2, 1, 8, 4, 7, 5, 3, 0, 9, 6]

In [52]:
import itertools

def online_random_sample(stream: Iterator[int], k: int) -> List[int]:
    running_sample = list(itertools.islice(stream, k))
    num_seen_so_far = k
    for x in stream:
        num_seen_so_far += 1
        idx_to_replace = random.randrange(num_seen_so_far)
        if idx_to_replace < k:
            running_sample[idx_to_replace] = x
    return running_sample

In [57]:
online_random_sample(list(range(10)), 10)

[0, 1, 3, 3, 2, 7, 6, 5, 8, 9]

In [56]:
def compute_random_permutation(n: int) -> List[int]:
    permutation = list(range(n))
    random_sampling(n, permutation)
    return permutation

In [59]:
compute_random_permutation(10)

[2, 3, 7, 5, 0, 4, 9, 8, 1, 6]

In [65]:
def random_subset(n: int, k: int) -> List[int]:
    changed_elements: Dict[int, int] = {}
    for i in range(k):
        rand_idx = random.randrange(i, n)
        rand_idx_mapped = changed_elements.get(rand_idx, rand_idx)
        i_mapped = changed_elements.get(i, i)
        print(rand_idx, rand_idx_mapped, i_mapped)
        changed_elements[rand_idx] = i_mapped
        changed_elements[i] = rand_idx_mapped
    return [changed_elements[i] for i in range(k)]

In [66]:
random_subset(10, 5)

5 5 0
6 6 1
7 7 2
4 4 3
7 2 3


[5, 6, 7, 4, 2]

In [68]:
import bisect

def nonuniform_random_number_generator(values: List[int], probabilities: List[float]) -> int:
    prefix_sum_of_probabilities = list(itertools.accumulate(probabilities))
    interval_idx = bisect.bisect(prefix_sum_of_probabilities, random.random())
    return values[interval_idx]

In [70]:
%timeit [nonuniform_random_number_generator([3, 5, 7, 11], [9/18, 6/18, 2/18, 1/18]) for _ in range(10000)]

12.6 ms ± 997 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [71]:
def is_valid_sudoku(partial_assignment: List[List[int]]) -> bool:
    def has_duplicate(block):
        block = list(filter(lambda x: x != 0, block))
        return len(block) != len(set(block))
    n = len(partial_assignment)
    if any(has_duplicate([partial_assignment[i][j] for j in range(n)])
           or has_duplicate([partial_assignment[j][i] for j in range(n)])
           for i in range(n)):
        return False
    region_size = int(math.sqrt(n))
    return all(not has_duplicate([partial_assignment[a][b] 
                                  for a in range(region_size * I, region_size * (I + 1)) 
                                  for b in range(region_size * J, region_size * (J + 1))
                                 ]) for I in range(region_size) for J in range(region_size))

In [129]:
partial_assignment = [[5,3,0,0,7,0,0,0,0],
                      [6,0,0,1,9,5,0,0,0],
                      [0,9,8,0,0,0,0,6,0],
                      
                      [8,0,0,0,6,0,0,0,3],
                      [4,0,0,8,0,3,0,0,1],
                      [7,0,0,0,2,0,0,0,6],
                      
                      [0,6,0,0,0,0,2,8,0],
                      [0,0,0,4,1,9,0,0,5],
                      [0,0,0,0,8,0,0,7,9]
                     ]

In [130]:
is_valid_sudoku(partial_assignment)

True

In [133]:
full_assignment = [[5,3,4,6,7,8,9,1,2],
                   [6,7,2,1,9,5,3,4,8],
                   [1,9,8,3,4,2,5,6,7],

                   [8,5,9,7,6,1,4,2,3],
                   [4,2,6,8,5,3,7,9,1],
                   [7,1,3,9,2,4,8,5,6],

                   [9,6,1,5,3,7,2,8,4],
                   [2,8,7,4,1,9,6,3,5],
                   [3,4,5,2,8,6,1,7,9]
                  ]

In [134]:
is_valid_sudoku(full_assignment)

True

In [127]:
def is_valid_sudoku_pythonic(partial_assignment):
    region_size = int(math.sqrt(len(partial_assignment)))
    return max(collections.Counter(k for i, row in enumerate(partial_assignment)
                                   for j, c in enumerate(row) if c != 0
                                   for k in ((i, str(c)), 
                                             (str(c), j), 
                                             (i // region_size, j // region_size, str(c)))).values(), 
               default=0) <= 1

In [128]:
is_valid_sudoku_pythonic(partial_assignment)

True

In [135]:
is_valid_sudoku_pythonic(full_assignment)

True

In [136]:
def is_valid_sudoku_pythonic(assignment):
    size = int(math.sqrt(len(assignment)))
    c = collections.Counter(k for i, row in enumerate(assignment)
                           for j, c in enumerate(row) if c != 0
                           for k in ((i, str(c)), (str(c), j), (i // size, j // size, str(c))))
    print(c)
    return max(c.values(), default=0) <= 1

In [137]:
is_valid_sudoku_pythonic(full_assignment)

Counter({(0, '5'): 1, ('5', 0): 1, (0, 0, '5'): 1, (0, '3'): 1, ('3', 1): 1, (0, 0, '3'): 1, (0, '4'): 1, ('4', 2): 1, (0, 0, '4'): 1, (0, '6'): 1, ('6', 3): 1, (0, 1, '6'): 1, (0, '7'): 1, ('7', 4): 1, (0, 1, '7'): 1, (0, '8'): 1, ('8', 5): 1, (0, 1, '8'): 1, (0, '9'): 1, ('9', 6): 1, (0, 2, '9'): 1, (0, '1'): 1, ('1', 7): 1, (0, 2, '1'): 1, (0, '2'): 1, ('2', 8): 1, (0, 2, '2'): 1, (1, '6'): 1, ('6', 0): 1, (0, 0, '6'): 1, (1, '7'): 1, ('7', 1): 1, (0, 0, '7'): 1, (1, '2'): 1, ('2', 2): 1, (0, 0, '2'): 1, (1, '1'): 1, ('1', 3): 1, (0, 1, '1'): 1, (1, '9'): 1, ('9', 4): 1, (0, 1, '9'): 1, (1, '5'): 1, ('5', 5): 1, (0, 1, '5'): 1, (1, '3'): 1, ('3', 6): 1, (0, 2, '3'): 1, (1, '4'): 1, ('4', 7): 1, (0, 2, '4'): 1, (1, '8'): 1, ('8', 8): 1, (0, 2, '8'): 1, (2, '1'): 1, ('1', 0): 1, (0, 0, '1'): 1, (2, '9'): 1, ('9', 1): 1, (0, 0, '9'): 1, (2, '8'): 1, ('8', 2): 1, (0, 0, '8'): 1, (2, '3'): 1, ('3', 3): 1, (0, 1, '3'): 1, (2, '4'): 1, ('4', 4): 1, (0, 1, '4'): 1, (2, '2'): 1, ('2', 5): 1,

True

In [138]:
full_assignment[0][0] += 1

In [139]:
is_valid_sudoku_pythonic(full_assignment)

Counter({(0, '6'): 2, ('6', 0): 2, (0, 0, '6'): 2, (0, '3'): 1, ('3', 1): 1, (0, 0, '3'): 1, (0, '4'): 1, ('4', 2): 1, (0, 0, '4'): 1, ('6', 3): 1, (0, 1, '6'): 1, (0, '7'): 1, ('7', 4): 1, (0, 1, '7'): 1, (0, '8'): 1, ('8', 5): 1, (0, 1, '8'): 1, (0, '9'): 1, ('9', 6): 1, (0, 2, '9'): 1, (0, '1'): 1, ('1', 7): 1, (0, 2, '1'): 1, (0, '2'): 1, ('2', 8): 1, (0, 2, '2'): 1, (1, '6'): 1, (1, '7'): 1, ('7', 1): 1, (0, 0, '7'): 1, (1, '2'): 1, ('2', 2): 1, (0, 0, '2'): 1, (1, '1'): 1, ('1', 3): 1, (0, 1, '1'): 1, (1, '9'): 1, ('9', 4): 1, (0, 1, '9'): 1, (1, '5'): 1, ('5', 5): 1, (0, 1, '5'): 1, (1, '3'): 1, ('3', 6): 1, (0, 2, '3'): 1, (1, '4'): 1, ('4', 7): 1, (0, 2, '4'): 1, (1, '8'): 1, ('8', 8): 1, (0, 2, '8'): 1, (2, '1'): 1, ('1', 0): 1, (0, 0, '1'): 1, (2, '9'): 1, ('9', 1): 1, (0, 0, '9'): 1, (2, '8'): 1, ('8', 2): 1, (0, 0, '8'): 1, (2, '3'): 1, ('3', 3): 1, (0, 1, '3'): 1, (2, '4'): 1, ('4', 4): 1, (0, 1, '4'): 1, (2, '2'): 1, ('2', 5): 1, (0, 1, '2'): 1, (2, '5'): 1, ('5', 6): 1,

False

In [145]:
def is_valid_sudoku_pythonic(assignment):
    size = int(math.sqrt(len(assignment)))
    c = Counter(k for i, row in enumerate(assignment)
                for j, c in enumerate(row) if c != 0
                for k in ((i, str(c)), (str(c), j), (i//size, j//size, str(c)))
               )
    print(c.most_common(1)[0][1])
    return max(c.values(),default=0) <= 1

In [146]:
is_valid_sudoku_pythonic(full_assignment)

2


False

In [153]:
def matrix_in_spiral_order(square_matrix: List[List[int]]) -> List[int]:
    def matrix_layer_in_clockwise(offset):
        if offset == len(square_matrix) - offset - 1:
            spiral_ordering.append(square_matrix[offset][offset])
            return
        spiral_ordering.extend(square_matrix[offset][offset:-1 - offset])
        spiral_ordering.extend(list(zip(*square_matrix))[-1 - offset][offset:-1 - offset])
        spiral_ordering.extend(square_matrix[-1 - offset][-1 - offset:offset:-1])
        spiral_ordering.extend(list(zip(*square_matrix))[offset][-1 - offset:offset:-1])
    spiral_ordering: List[int] = []
    for offset in range((len(square_matrix) + 1) // 2):
        matrix_layer_in_clockwise(offset)
    return spiral_ordering

In [151]:
x1 = np.array(range(9)).reshape(3,3)
x1

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [152]:
x2 = np.array(range(16)).reshape(4,4)
x2

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [154]:
matrix_in_spiral_order(x1)

[0, 1, 2, 5, 8, 7, 6, 3, 4]

In [155]:
matrix_in_spiral_order(x2)

[0, 1, 2, 3, 7, 11, 15, 14, 13, 12, 8, 4, 5, 6, 10, 9]